In [1]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import glob
import re
#import xmltodict
import pickle
import untangle
import uuid
from tqdm import tqdm
from decimal import Decimal
from skimage import measure, morphology
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.segmentation import clear_border
from skimage.filters import roberts, sobel

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

DATA_PATH = '/kaggle_2/lidc_idri/data/'
DATA_PATH_XML = '/kaggle_2/lidc_idri/data/tcia-lidc-xml/'
DATA_PATH_SCANS = '/kaggle_2/lidc_idri/data/LIDC/DOI/'
DATA_PATH_POST_PROCESSED_SCANS = '/kaggle_2/lidc_idri/data/scans_resampled_unsegmented/'
DATA_PATH_NODULES = '/kaggle_2/lidc_idri/data/nodules_chunked/'
DATA_PATH_NON_NODULES = '/kaggle_2/lidc_idri/data/non_nodules_chunked/'
CHUNK_SIZE = 32

In [10]:
with open(DATA_PATH + 'patient_scans_map.pkl', 'rb') as f:
    patient_scans_map = pickle.load(f)

with open(DATA_PATH + 'patient_nodules_map.pkl', 'rb') as f:
    patient_nodules_map = pickle.load(f)
    
with open(DATA_PATH + 'patient_non_nodules_map.pkl', 'rb') as f:
    patient_non_nodules_map = pickle.load(f)

In [11]:
print(len(patient_scans_map.keys()))
print(len(patient_nodules_map.keys()))
print(len(patient_non_nodules_map.keys()))

1018
883
128


In [12]:
# Load the scans in given folder path
def load_scan(paths):
    slices = [dicom.read_file(path) for path in paths]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
    
    origin = np.array(list(reversed(slices[0].ImagePositionPatient)), dtype=np.float32)
    
    # Determine current pixel spacing
    spacing = np.array([slices[0].SliceThickness] + slices[0].PixelSpacing, dtype=np.float32)

    return slices, origin, spacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

In [22]:
weird_chunks = {}
weird_chunk_count = 0
RESIZE_SPACING = [1,1,1]
diameters = []
items = list(patient_nodules_map.items())
for idx in tqdm(range(len(items))):
    patient_id = items[idx][0]
    nodules = items[idx][1]
#     print(nodules)
#     print(patient_id)
#     patient_scan_files = patient_scans_map[patient_id]['scans']
#     patient_scan_files.sort()
#     scan, origin, spacing = load_scan(patient_scan_files)
    scan_resampled = np.load(DATA_PATH_POST_PROCESSED_SCANS + "scan_%s.npy" % (patient_id))
#     print('Original scan', (len(scan), 512, 512))
#     print('Resampled scan', scan_resampled.shape)
#     print('Nodules',len(nodules))
#     print(nodules)
#     print('---')
#     print(patient_nodules_map[patient_id])

    X = []#np.ndarray((len(nodules), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), dtype=np.float32)
    Y = []#np.ndarray([len(nodules), 1], dtype=np.float32)
    count = 0
    for nodule in nodules:
        coordsPerc = nodule['coordsPerc']
       
        centerZ = int(coordsPerc[0] * scan_resampled.shape[0])
        centerY = int(coordsPerc[1] * scan_resampled.shape[1])
        centerX = int(coordsPerc[2] * scan_resampled.shape[2])
        diameter = float(nodule['diameterPerc'] * scan_resampled.shape[2])
        
        assert scan_resampled.shape[1] == scan_resampled.shape[2]
        
        diameters.append(diameter)
        continue
        
        Z1 = centerZ - int(CHUNK_SIZE/2)
        Z2 = centerZ + int(CHUNK_SIZE/2)
        Y1 = centerY - int(CHUNK_SIZE/2)
        Y2 = centerY + int(CHUNK_SIZE/2)
        X1 = centerX - int(CHUNK_SIZE/2)
        X2 = centerX + int(CHUNK_SIZE/2)
        
        X1 = 0 if (X1 < 0) else X1
        Y1 = 0 if (Y1 < 0) else Y1
        Z1 = 0 if (Z1 < 0) else Z1
        
        X2 = scan_resampled.shape[2] if (X2 > scan_resampled.shape[2]) else X2
        Y2 = scan_resampled.shape[1] if (Y2 > scan_resampled.shape[1]) else Y2
        Z2 = scan_resampled.shape[0] if (Z2 > scan_resampled.shape[0]) else Z2
        
#         print(int(minZ), int(maxZ), int(minY), int(maxY), int(minX), int(maxX))
#         print(centerZ, centerY, centerX)
#         print(Z1, Z2, Y1, Y2, X1, X2)
        
        if (Z2 > scan_resampled.shape[0] or Y2 > scan_resampled.shape[1] or X2 > scan_resampled.shape[2] or Z2 < Z1 or Y2 < Y1 or X2 < X1):
            #print('Found weird chunk!')
            if patient_id in weird_chunks.keys():
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            else:
                weird_chunks[patient_id] = []
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            weird_chunk_count += 1
            continue
        
        chunk = np.full((CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), -1000.0, np.float32)
        chunk[0:Z2-Z1, 0:Y2-Y1, 0:X2-X1] = scan_resampled[Z1:Z2,Y1:Y2,X1:X2]

        X.append(chunk)
        if 'malignancy' in nodule:
            Y.append(nodule['malignancy'])
        else:
            Y.append(0.0)
        count = count + 1
    
    assert len(X) == len(Y)
    
#     X_np = np.ndarray((len(X), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), dtype=np.float32)
#     Y_np = np.ndarray([len(X)], dtype=np.float32)
    
#     for idx in range(len(X)):
#         X_np[idx, :, :, :] = X[idx]
#         Y_np[idx] = Y[idx]
        
#     np.save(DATA_PATH_NODULES + patient_id + '_X.npy', X_np)
#     np.save(DATA_PATH_NODULES + patient_id + '_Y.npy', Y_np)


100%|██████████| 883/883 [55:44<00:00,  6.99s/it]  


In [28]:
diameters_np = np.asarray(diameters)
for i in range(90,100):
    print('percentile', i, ':', np.percentile(diameters_np, i))
    print('percentile', i+0.5, ':', np.percentile(diameters_np, i+0.5))

percentile 90 : 21.28848
percentile 90.5 : 21.88
percentile 91 : 22.502146
percentile 91.5 : 23.188
percentile 92 : 23.44
percentile 92.5 : 24.068
percentile 93 : 24.90505
percentile 93.5 : 25.7298
percentile 94 : 26.476552
percentile 94.5 : 27.37656
percentile 95 : 28.21686
percentile 95.5 : 28.92
percentile 96 : 30.24
percentile 96.5 : 30.924
percentile 97 : 32.328
percentile 97.5 : 33.768
percentile 98 : 35.685568
percentile 98.5 : 37.150095
percentile 99 : 39.27178
percentile 99.5 : 43.476846


In [12]:
# Do NOT run this cell as it will overwrite already existing weird_chunks file on disk
with open(DATA_PATH + 'weird_chunks.pkl', 'wb') as f:
    pickle.dump(weird_chunks, f, pickle.HIGHEST_PROTOCOL)
    
print('Saved weird_chunks map!')

Saved weird_chunks map!


In [13]:
with open(DATA_PATH + 'weird_chunks.pkl', 'rb') as f:
    weird_chunks = pickle.load(f)

In [21]:
weird_chunks = {}
weird_chunk_count = 0
RESIZE_SPACING = [1,1,1]
items = list(patient_non_nodules_map.items())
for idx in tqdm(range(len(items))):
    patient_id = items[idx][0]
    nodules = items[idx][1]
    #print(nodules)
#   print(patient_id)
    #patient_scan_files = patient_scans_map[patient_id]['scans']
    #patient_scan_files.sort()
    #scan, origin, spacing = load_scan(patient_scan_files)
    scan_resampled = np.load(DATA_PATH_POST_PROCESSED_SCANS + "scan_%s.npy" % (patient_id))
#    print('Original scan', (len(scan), 512, 512))
#    print('Resampled scan', scan_resampled.shape)
#    print('Nodules',len(nodules))
    #print(nodules)
    #print('---')
    #print(patient_nodules_map[patient_id])

    X = []#np.ndarray((len(nodules), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), dtype=np.float32)
    Y = []#np.ndarray([len(nodules), 1], dtype=np.float32)
    count = 0
    for nodule in nodules:
        coordsPerc = nodule['coordsPerc']
       
        centerZ = int(coordsPerc[0] * scan_resampled.shape[0])
        centerY = int(coordsPerc[1] * scan_resampled.shape[1])
        centerX = int(coordsPerc[2] * scan_resampled.shape[2])
        
        Z1 = centerZ - int(CHUNK_SIZE/2)
        Z2 = centerZ + int(CHUNK_SIZE/2)
        Y1 = centerY - int(CHUNK_SIZE/2)
        Y2 = centerY + int(CHUNK_SIZE/2)
        X1 = centerX - int(CHUNK_SIZE/2)
        X2 = centerX + int(CHUNK_SIZE/2)
        
        X1 = 0 if (X1 < 0) else X1
        Y1 = 0 if (Y1 < 0) else Y1
        Z1 = 0 if (Z1 < 0) else Z1
        
        X2 = scan_resampled.shape[2] if (X2 > scan_resampled.shape[2]) else X2
        Y2 = scan_resampled.shape[1] if (Y2 > scan_resampled.shape[1]) else Y2
        Z2 = scan_resampled.shape[0] if (Z2 > scan_resampled.shape[0]) else Z2
        
#      print(int(minZ), int(maxZ), int(minY), int(maxY), int(minX), int(maxX))
#         print(centerZ, centerY, centerX)
#         print(Z1, Z2, Y1, Y2, X1, X2)
        
        if (Z2 > scan_resampled.shape[0] or Y2 > scan_resampled.shape[1] or X2 > scan_resampled.shape[2] or Z2 < Z1 or Y2 < Y1 or X2 < X1):
            #print('Found weird chunk!')
            if patient_id in weird_chunks.keys():
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            else:
                weird_chunks[patient_id] = []
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            weird_chunk_count += 1
            continue
        
        chunk = np.full((CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), -1000.0, np.float32)
        chunk[0:Z2-Z1, 0:Y2-Y1, 0:X2-X1] = scan_resampled[Z1:Z2,Y1:Y2,X1:X2]

        X.append(chunk)
        if 'malignancy' in nodule:
            Y.append(nodule['malignancy'])
        else:
            Y.append(0.0)
        count = count + 1
    
    assert len(X) == len(Y)
    
    X_np = np.ndarray((len(X), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), dtype=np.float32)
    Y_np = np.ndarray([len(X)], dtype=np.float32)
    
    for idx in range(len(X)):
        X_np[idx, :, :, :] = X[idx]
        Y_np[idx] = Y[idx]
        
    np.save(DATA_PATH_NON_NODULES + patient_id + '_X.npy', X_np)
    np.save(DATA_PATH_NON_NODULES + patient_id + '_Y.npy', Y_np)

100%|██████████| 128/128 [04:58<00:00,  2.60s/it]


In [24]:
non_nodules_count = 0
for p_id, nodules in patient_non_nodules_map.items():
    non_nodules_count += len(nodules)
non_nodules_count

1373